TODO:
- add None into test dataset

In [1]:
import os, time
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import csv
from PIL import Image
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import VisionDataset

In [3]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        img = img.convert('RGB')
    return img

class DeepFashion(VisionDataset):
    def __init__(self, csv_file, mode, transform):
        self.mode = mode # train, test and val
        self.transform = transform
#         self.target_transform = target_transform
        
        img_list = []
        cate_list = []
        attr_list = []
        with open(csv_file) as f:
            reader = csv.DictReader(f)
            for line in reader:
                img_list.append(line['file_path'])
                if mode != 'test':
                    cate = int(line['category_label'])
                    cate_list.append(cate)
                    
                    attrs = tuple(line.values())[2:]
                    attrs = [int(a) for a in attrs]
                    attr_list.append(attrs) # save using tuple
        self.img_list = img_list
        self.cate_list = cate_list
        self.attr_list = attr_list
    
    def __getitem__(self, idx):
        path = self.img_list[idx]
        img = pil_loader(path)
        if self.transform:
            img = self.transform(img)

        if self.mode != 'test':
            category = self.cate_list[idx]
            category = torch.tensor(category)
            attribute = torch.tensor(self.attr_list[idx])
            return img, category, attribute
        else:
            return img
    
    def __len__(self):
        return len(self.img_list)

In [4]:
trans_train = transforms.Compose([
    transforms.Resize(240),
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
trans_test = transforms.Compose([
    transforms.Resize(240),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
# target_trans = {32: 0, 33: 1, 3: 2, 6: 3, 41: 4,
#                 11: 5, 16: 6, 17: 7, 18: 8, 19: 9}

train_dataset = DeepFashion('deep_fashion/train.csv', 'train', trans_train)
val_dataset = DeepFashion('deep_fashion/val.csv', 'val', trans_test)
test_dataset = DeepFashion('deep_fashion/test.csv', 'test', trans_test)

In [5]:
print(len(train_dataset.img_list))
print(len(train_dataset.cate_list))
print(len(train_dataset.attr_list))

39092
39092
39092


In [6]:
print('Train', len(train_dataset))
print('Val', len(val_dataset))
print('Test', len(test_dataset))

Train 39092
Val 5528
Test 11225


In [7]:
batch_size = 64
worker = 8
train_data = DataLoader(train_dataset, batch_size=batch_size,
                        num_workers=worker, pin_memory=True,
                        shuffle=True)
val_data = DataLoader(val_dataset, batch_size=batch_size,
                        num_workers=worker, pin_memory=True)
test_data = DataLoader(test_dataset, batch_size=batch_size,
                        num_workers=worker, pin_memory=True)

In [8]:
# next(iter(train_data))

In [9]:
# SENet 1_1
model = models.squeezenet1_1(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

for param in model.features[-1].parameters():
    param.requires_grad = True

# v2
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(64, 10, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1, 1)),
)

In [10]:
## resnet50
# model = models.resnext50_32x4d(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False
# in_features = model.fc.in_features
# model.fc = nn.Linear(in_features, 10)

In [11]:
device = torch.device('cuda')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=3e-5)
criterion = nn.CrossEntropyLoss().to(device)

In [12]:
def train(model, data, epoch, criterion, optimizer, device):
    model.train()
    print('==========Train Epoch {}=========='.format(epoch))
    loss_list = []
    acc_count = 0
    total_count = 0

    for i, (image, label, _) in tqdm(enumerate(data), ascii=True, total=len(data)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        score = model(image) # predict the label
        loss = criterion(score, label) # calculate error
        loss_list.append(loss.item())

        pred = torch.argmax(score, dim=1)
        correct = pred.eq(label)
        acc_count += correct.sum().item()
        total_count += image.shape[0]
        
        loss.backward()  # back-propagation
        optimizer.step() # gradient descent

    acc = acc_count / total_count * 100
    return sum(loss_list) / len(loss_list), acc

def test(model, data, criterion, device):
    model.eval()
    loss_list = []
    acc_count = 0
    total_count = 0

    for i, (image, label, _) in tqdm(enumerate(data), ascii=True, total=len(data)):
        image = image.to(device)
        label = label.to(device)

        score = model(image)
        loss = criterion(score, label)
        loss_list.append(loss.item())

        pred = torch.argmax(score, dim=1)
        correct = pred.eq(label)
        acc_count += correct.sum().item()
        total_count += image.shape[0]

    acc = acc_count / total_count * 100
    print('----------Acc: {}%----------'.format(acc))
    return sum(loss_list) / len(loss_list), acc

In [13]:
# Hyper Parameters
max_epochs = 5
log_interval = 1

train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []

for epoch in range(1, max_epochs + 1):
    t = time.time()
    train_loss, train_acc = train(model, train_data, epoch, criterion, optimizer, device)
    val_loss, val_acc = test(model, val_data, criterion, device)
    print('Cost', time.time() - t, 'secs')

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)
    if epoch % log_interval == 0:
#         print('=' * 20, 'Epoch', epoch, '=' * 20)
        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss))
        print('Test Acc: {:.6f} Test Loss: {:.6f}'.format(val_acc, val_loss))

==========Train Epoch 1==========


100%|##########| 87/87 [00:03<00:00, 25.89it/s]

----------Acc: 30.17366136034732%----------
Cost 31.867602348327637 secs
Train Acc: 21.285685 Train Loss: 2.182107
Test Acc: 30.173661 Test Loss: 2.012580
==========Train Epoch 2==========



100%|##########| 87/87 [00:03<00:00, 26.57it/s]

----------Acc: 35.07597684515195%----------
Cost 31.084635496139526 secs
Train Acc: 31.198199 Train Loss: 1.953731
Test Acc: 35.075977 Test Loss: 1.887717
==========Train Epoch 3==========


...

----------Acc: 37.44573082489146%----------
Cost 30.367223024368286 secs
Train Acc: 33.679525 Train Loss: 1.888553
Test Acc: 37.445731 Test Loss: 1.813228
==========Train Epoch 4==========



100%|##########| 87/87 [00:03<00:00, 26.88it/s]

----------Acc: 38.83863965267728%----------
Cost 31.49911594390869 secs
Train Acc: 35.019953 Train Loss: 1.850375
Test Acc: 38.838640 Test Loss: 1.769724
==========Train Epoch 5==========



100%|##########| 87/87 [00:03<00:00, 25.31it/s]

----------Acc: 40.73806078147612%----------
Cost 30.724737882614136 secs
Train Acc: 36.275964 Train Loss: 1.824129
Test Acc: 40.738061 Test Loss: 1.728970
